In [1]:
import pandas as pd

## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Download the data for January and February 2022.

Read the data for January. How many columns are there?

* 16
* 17
* 18
* 19 &check;


In [2]:
# ! wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet
# ! wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet

In [3]:
df = pd.read_parquet('./yellow_tripdata_2022-01.parquet')

In [4]:
df.shape

(2463931, 19)

## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the standard deviation of the trips duration in January?

* 41.45
* 46.45 &check;
* 51.45
* 56.45

In [5]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
df.duration.std()

46.44530513776802

## Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

* 90%
* 92%
* 95%
* 98% &check;

In [6]:
old_size = df.shape[0]
df = df[(df.duration >= 1) & (df.duration <= 60)]
new_size = df.shape[0]
new_size / old_size * 100

98.27547930522405

## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

* 2
* 155
* 345
* 515 &check;
* 715

In [7]:
from sklearn.feature_extraction import DictVectorizer

In [8]:
y_train = df.duration

train_dicts = df[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_train.shape[1]

515

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters 
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

* 6.99 &check;
* 11.99
* 16.99
* 21.99

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [10]:
model = LinearRegression()
y_pred = model.fit(X_train, y_train).predict(X_train)
mean_squared_error(y_train, y_pred, squared=False)

6.986190814952337

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2022). 

What's the RMSE on validation?

* 7.79 &check;
* 12.79
* 17.79
* 22.79

In [11]:
df = pd.read_parquet('./yellow_tripdata_2022-02.parquet')
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
df = df[(df.duration >= 1) & (df.duration <= 60)]

y_test = df.duration
test_dicts = df[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')
X_test = dv.transform(test_dicts)

y_pred = model.predict(X_test)
mean_squared_error(y_test, y_pred, squared=False)

7.786407163179794